In [1]:
import yaml
import os
import json
from tqdm import tqdm
from PIL import Image
import shutil
import time
import torch

In [2]:
# 1️⃣ Set Up Working Directory
# -----------------------
root_dir = "/content"  # Colab default working directory
os.chdir(root_dir)
print(f"Working directory set to: {os.getcwd()}")


Working directory set to: /content


In [5]:
# 2️⃣ Clone NanoDet Repo
!rm -rf nanodet
!git clone https://github.com/RangiLyu/nanodet.git
%cd nanodet/

print("Repo cloned successfully! Directory contents:")
!ls -F

print("Config directories:")
!ls -F config/
!ls -F config/legacy_v0.x_configs/

Cloning into 'nanodet'...
remote: Enumerating objects: 2722, done.
remote: Total 2722 (delta 0), reused 0 (delta 0), pack-reused 2722 (from 1)
Receiving objects: 100% (2722/2722), 5.29 MiB | 34.30 MiB/s, done.
Resolving deltas: 100% (1602/1602), done.
/content/nanodet/nanodet
Repo cloned successfully! Directory contents:
config/		    demo_libtorch/  demo_openvino/  nanodet/	      setup.py
demo/		    demo_mnn/	    docs/	    README.md	      tests/
demo_android_ncnn/  demo_ncnn/	    LICENSE	    requirements.txt  tools/
Config directories:
convnext/			nanodet-plus-m-1.5x_416.yml
legacy_v0.x_configs/		nanodet-plus-m_320.yml
nanodet_custom_xml_dataset.yml	nanodet-plus-m_416.yml
nanodet-plus-m-1.5x_320.yml	nanodet-plus-m_416-yolo.yml
EfficientNet-Lite/  nanodet-m-1.5x-416.yml  nanodet-m.yml
nanodet-g.yml	    nanodet-m-1.5x.yml	    RepVGG/
nanodet-m-0.5x.yml  nanodet-m-416.yml	    Transformer/


In [7]:
!find /content/nanodet -type f -name "collate.py"

/content/nanodet/nanodet/nanodet/data/collate.py


In [8]:
# 3️⃣ Apply Patches
# -----------------------
!sed -i "19s/from torch._six import string_classes/string_classes = str/" /content/nanodet/nanodet/nanodet/data/collate.py
!sed -i "146s/strategy=strategy,/strategy='auto',/" /content/nanodet/tools/train.py
!sed -i "83s/ckpt = convert_old_model(ckpt)/# ckpt = convert_old_model(ckpt) # Patched/" /content/nanodet/tools/test.py
print("✅ All patches applied successfully!")


✅ All patches applied successfully!


In [9]:
# 4️⃣ Install Dependencies
# -----------------------
!pip install -q pyyaml opencv-python tqdm tensorboard torchmetrics pycocotools
!pip install -q -e .

print("\n✅ Environment setup complete!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

✅ Environment setup complete!


In [16]:
!unzip /content/hit-uav-dataset.zip -d /content/hit-uav

Streaming output truncated to the last 5000 lines.
  inflating: /content/hit-uav/hit-uav/images/train/0_110_70_0_08486.jpg  
  inflating: /content/hit-uav/hit-uav/images/train/0_110_80_0_03729.jpg  
  inflating: /content/hit-uav/hit-uav/images/train/0_110_80_0_03734.jpg  
  inflating: /content/hit-uav/hit-uav/images/train/0_110_80_0_08491.jpg  
  inflating: /content/hit-uav/hit-uav/images/train/0_110_80_0_08512.jpg  
  inflating: /content/hit-uav/hit-uav/images/train/0_110_80_0_08518.jpg  
  inflating: /content/hit-uav/hit-uav/images/train/0_110_80_0_08520.jpg  
  inflating: /content/hit-uav/hit-uav/images/train/0_110_80_0_08522.jpg  
  inflating: /content/hit-uav/hit-uav/images/train/0_110_90_0_03784.jpg  
  inflating: /content/hit-uav/hit-uav/images/train/0_110_90_0_03802.jpg  
  inflating: /content/hit-uav/hit-uav/images/train/0_110_90_0_08527.jpg  
  inflating: /content/hit-uav/hit-uav/images/train/0_110_90_0_08529.jpg  
  inflating: /content/hit-uav/hit-uav/images/train/0_110_90_0

In [17]:
# 5️⃣ Dataset Setup
# -----------------------
# Upload your dataset to Google Drive or Colab environment
# Example: /content/hit-uav with images/ and labels/ folders
dataset_root = "/content/hit-uav"  # <-- adjust this if uploaded elsewhere

os.makedirs(dataset_root, exist_ok=True)

# Create dataset.yaml
dataset_yaml_path = os.path.join(root_dir, "dataset.yaml")
new_yaml_content = """
path: /content/hit-uav
train: /content/hit-uav/images/train
val: /content/hit-uav/images/val
test: /content/hit-uav/images/test
names:
  0: Person
  1: Car
nc: 2
"""
with open(dataset_yaml_path, "w") as f:
    f.write(new_yaml_content)
print("✅ dataset.yaml created at:", dataset_yaml_path)


✅ dataset.yaml created at: /content/dataset.yaml


In [24]:
import os, json, shutil, yaml
from PIL import Image
from tqdm import tqdm

def convert_yolo_to_coco(dataset_root_path, output_dir):
    print("Converting YOLO dataset to COCO format...")
    os.makedirs(os.path.join(output_dir, 'annotations'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'test'), exist_ok=True)

    yaml_path = os.path.join(dataset_root_path, "dataset.yaml")
    with open(yaml_path, "r") as f:
        data_yaml = yaml.safe_load(f)
    class_names = data_yaml['names']

    split_map = {"train":"train", "test":"test", "val":"val"}  # map folder names to COCO splits

    for folder, split in split_map.items():
        print(f"Processing {folder} -> {split} split...")
        image_dir = os.path.join(dataset_root_path, 'images', folder)
        label_dir = os.path.join(dataset_root_path, 'labels', folder)
        coco_output = {
            "info": {}, "licenses": [],
            "categories": [{"id": i, "name": name, "supercategory": "object"} for i, name in enumerate(class_names)],
            "images": [], "annotations": []
        }

        if not os.path.exists(image_dir):
            print(f"⚠️ No images found for {folder}, skipping.")
            continue

        image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        image_id, ann_id = 0, 0
        for img_file in tqdm(image_files):
            img_path = os.path.join(image_dir, img_file)
            with Image.open(img_path) as img:
                w, h = img.size
            image_info = {"id": image_id, "file_name": img_file, "width": w, "height": h}
            coco_output["images"].append(image_info)

            label_file = os.path.splitext(img_file)[0] + ".txt"
            label_path = os.path.join(label_dir, label_file)
            if os.path.exists(label_path):
                with open(label_path, "r") as f:
                    for line in f:
                        cls, x, y, bw, bh = map(float, line.strip().split())
                        x_min, y_min = (x - bw / 2) * w, (y - bh / 2) * h
                        coco_output["annotations"].append({
                            "id": ann_id, "image_id": image_id,
                            "category_id": int(cls),
                            "bbox": [x_min, y_min, bw * w, bh * h],
                            "area": bw * bh * w * h, "iscrowd": 0
                        })
                        ann_id += 1
            shutil.copy(img_path, os.path.join(output_dir, split, img_file))
            image_id += 1

        with open(os.path.join(output_dir, "annotations", f"{split}.json"), "w") as f:
            json.dump(coco_output, f)
        print(f"✅ {split}.json saved!")

# Usage
yolo_dataset_path = "/content/hit-uav/hit-uav"
coco_output_path = "/content/hituav_coco"
convert_yolo_to_coco(yolo_dataset_path, coco_output_path)
print("✅ Dataset conversion complete!")


Converting YOLO dataset to COCO format...
Processing train -> train split...


100%|██████████| 2008/2008 [00:00<00:00, 2635.98it/s]


✅ train.json saved!
Processing test -> test split...


100%|██████████| 571/571 [00:00<00:00, 2786.28it/s]


✅ test.json saved!
Processing val -> val split...


100%|██████████| 287/287 [00:00<00:00, 3060.63it/s]

✅ val.json saved!
✅ Dataset conversion complete!


In [23]:
rm -rf /content/hituav_coco

In [25]:
# 7️⃣ Modify Config for Custom Dataset
# -----------------------
config_template = "config/legacy_v0.x_configs/nanodet-m.yml"
custom_config = "config/nanodet_hituav.yml"
!cp {config_template} {custom_config}

with open(custom_config, "r") as f:
    cfg = yaml.safe_load(f)

cfg['save_dir'] = '/content/model_workspace/'
cfg['class_names'] = ["Person", "Car"]
cfg['model']['arch']['head']['num_classes'] = 2
cfg['data']['train']['img_path'] = '/content/hituav_coco/train'
cfg['data']['train']['ann_path'] = '/content/hituav_coco/annotations/train.json'
cfg['data']['val']['img_path'] = '/content/hituav_coco/val'
cfg['data']['val']['ann_path'] = '/content/hituav_coco/annotations/val.json'
cfg['data']['test'] = {
    'name': 'CocoDataset',
    'img_path': '/content/hituav_coco/test',
    'ann_path': '/content/hituav_coco/annotations/test.json',
    'input_size': [320, 320],
    'keep_ratio': True
}
cfg['data']['train']['input_size'] = [320, 320]
cfg['data']['val']['input_size'] = [320, 320]
cfg['data']['dataloader_cfg'] = {'num_workers': 2}
cfg['device']['batch_size_per_gpu'] = 16
cfg['schedule']['total_epochs'] = 50

with open(custom_config, "w") as f:
    yaml.dump(cfg, f)
print("✅ Custom config saved!")


✅ Custom config saved!


In [15]:
# 8️⃣ Install Compatible PyTorch Lightning
# -----------------------
!pip uninstall -y pytorch-lightning
!pip install pytorch-lightning==1.9.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 21.2 MB/s eta 0:00:00


In [26]:
# 9️⃣ Clear CUDA Cache
# -----------------------
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU Ready: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU not available. Training will be on CPU.")

GPU Ready: Tesla T4


In [27]:
# 🔟 Train the Model
# -----------------------
print("\n--- Starting Training ---")
!python tools/train.py config/nanodet_hituav.yml


--- Starting Training ---
[NanoDet][10-17 10:41:30]INFO:Setting up data...
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
[NanoDet][10-17 10:41:30]INFO:Creating model...
model size is  1.0x
init weights...
Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x1-5666bf0f80.pth
100% 8.79M/8.79M [00:00<00:00, 105MB/s]
=> loading pretrained model 

In [29]:
!zip -r /content/nanodet_model_workspace.zip /content/model_workspace

  adding: content/model_workspace/ (stored 0%)
  adding: content/model_workspace/logs-2025-10-17-10-41-30/ (stored 0%)
  adding: content/model_workspace/logs-2025-10-17-10-41-30/Train_loss_lr_Train/ (stored 0%)
  adding: content/model_workspace/logs-2025-10-17-10-41-30/Train_loss_lr_Train/events.out.tfevents.1760697709.b1941aebac95.6479.1 (deflated 63%)
  adding: content/model_workspace/logs-2025-10-17-10-41-30/logs.txt (deflated 84%)
  adding: content/model_workspace/logs-2025-10-17-10-41-30/Val_metrics_AP_small_Val/ (stored 0%)
  adding: content/model_workspace/logs-2025-10-17-10-41-30/Val_metrics_AP_small_Val/events.out.tfevents.1760697935.b1941aebac95.6479.8 (deflated 44%)
  adding: content/model_workspace/logs-2025-10-17-10-41-30/Val_metrics_AP_75_Val/ (stored 0%)
  adding: content/model_workspace/logs-2025-10-17-10-41-30/Val_metrics_AP_75_Val/events.out.tfevents.1760697935.b1941aebac95.6479.7 (deflated 43%)
  adding: content/model_workspace/logs-2025-10-17-10-41-30/Train_loss_los